In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split, Subset
from sklearn.preprocessing import MinMaxScaler
from util.env import get_device, set_device
from util.preprocess import build_loc_net, construct_data
from util.net_struct import get_feature_map, get_fc_graph_struc
from util.iostream import printsep
from datasets.TimeDataset import TimeDataset
from models.GDN import GDN
from train_model import train_model
from test_model  import test_model
from evaluate import get_err_scores, get_best_performance_data, get_val_performance_data, get_full_err_scores
import sys
from datetime import datetime
import os
import argparse
from pathlib import Path
import matplotlib.pyplot as plt
import json
import random

class Args:
    def __init__(self):
        self.batch = 32
        self.epoch = 50
        self.slide_win = 5
        self.dim = 64
        self.slide_stride = 5
        self.save_path_pattern = ''
        self.dataset = 'swat'
        self.device = 'cuda'
        self.random_seed = 0
        self.comment = ''
        self.out_layer_num = 1
        self.out_layer_inter_dim = 64
        self.decay = 0.0
        self.val_ratio = 0.1
        self.topk = 15
        self.report = 'best'
        self.load_model_path = ''

        # 추가된 변수들
        self.npy_save_path = './eval_loss_acc.npy'  # Path to save evaluation results
        self.pool_save_path = './prompt_pool.npy'  # Path to save prompt pool
        self.dataset_root = './Dataset'  # Root directory for datasets

args = Args()
print(args.device)

random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.manual_seed(args.random_seed)
torch.cuda.manual_seed(args.random_seed)
torch.cuda.manual_seed_all(args.random_seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
os.environ['PYTHONHASHSEED'] = str(args.random_seed)

cuda


In [2]:
train_config = {
    'batch': args.batch,
    'epoch': args.epoch,
    'slide_win': args.slide_win,
    'dim': args.dim,
    'slide_stride': args.slide_stride,
    'comment': args.comment,
    'seed': args.random_seed,
    'out_layer_num': args.out_layer_num,
    'out_layer_inter_dim': args.out_layer_inter_dim,
    'decay': args.decay,
    'val_ratio': args.val_ratio,
    'topk': args.topk,
}

env_config={
    'save_path': args.save_path_pattern,
    'dataset': args.dataset,
    'report': args.report,
    'device': args.device,
    'load_model_path': args.load_model_path
}

In [3]:
datestr = None

dataset = env_config['dataset'] 
train_orig = pd.read_csv(f'./data/{dataset}/train.csv', sep=',', index_col=0)
test_orig = pd.read_csv(f'./data/{dataset}/test.csv', sep=',', index_col=0)

train, test = train_orig, test_orig

if 'attack' in train.columns:
    train = train.drop(columns=['attack'])

feature_map = get_feature_map(dataset)
# Create Fully-Connected Graph Structure
fc_struc = get_fc_graph_struc(dataset)

set_device(env_config['device'])
device = get_device()

In [4]:
# Create Edge Connections
fc_edge_index = build_loc_net(fc_struc, list(train.columns), feature_map=feature_map)
fc_edge_index = torch.tensor(fc_edge_index, dtype = torch.long)

feature_map = feature_map

# Data List & Label List
train_dataset_indata = construct_data(train, feature_map, labels=0)
test_dataset_indata = construct_data(test, feature_map, labels=test.attack.tolist())


cfg = {
    'slide_win': train_config['slide_win'],
    'slide_stride': train_config['slide_stride'],
}

train_dataset = TimeDataset(train_dataset_indata, fc_edge_index, mode='train', config=cfg)
test_dataset = TimeDataset(test_dataset_indata, fc_edge_index, mode='test', config=cfg)

In [5]:
train_dataset.x.shape, train_dataset.y.shape, train_dataset.labels.shape

(torch.Size([9467, 51, 5]), torch.Size([9467, 51]), torch.Size([9467]))

In [6]:
def get_loaders(train_dataset, seed, batch, val_ratio=0.1):
    dataset_len = int(len(train_dataset))
    train_use_len = int(dataset_len * (1 - val_ratio))
    val_use_len = int(dataset_len * val_ratio)
    val_start_index = random.randrange(train_use_len)
    indices = torch.arange(dataset_len)

    train_sub_indices = torch.cat([indices[:val_start_index], indices[val_start_index+val_use_len:]])
    train_subset = Subset(train_dataset, train_sub_indices)

    val_sub_indices = indices[val_start_index:val_start_index+val_use_len]
    val_subset = Subset(train_dataset, val_sub_indices)


    train_dataloader = DataLoader(train_subset, batch_size=batch,
                            shuffle=True)

    val_dataloader = DataLoader(val_subset, batch_size=batch,
                            shuffle=False)

    return train_dataloader, val_dataloader

train_dataloader, val_dataloader = get_loaders(train_dataset, train_config['seed'], train_config['batch'], val_ratio = train_config['val_ratio'])

train_dataset = train_dataset
test_dataset = test_dataset


train_dataloader = train_dataloader
val_dataloader = val_dataloader
test_dataloader = DataLoader(test_dataset, batch_size=train_config['batch'],
                    shuffle=False, num_workers=0)


edge_index_sets = []
edge_index_sets.append(fc_edge_index)

model = GDN(edge_index_sets, len(feature_map), 
        dim=train_config['dim'], 
        input_dim=train_config['slide_win'],
        out_layer_num=train_config['out_layer_num'],
        out_layer_inter_dim=train_config['out_layer_inter_dim'],
        topk=train_config['topk']
    ).to(device)


In [ ]:
# x, y, label, edge
next(iter(train_dataloader))[0].shape, next(iter(train_dataloader))[1].shape, next(iter(train_dataloader))[2].shape, next(iter(train_dataloader))[3].shape

(torch.Size([32, 51, 5]),
 torch.Size([32, 51]),
 torch.Size([32]),
 torch.Size([32, 2, 2550]))

In [19]:
def get_save_path(feature_name=''):

    dir_path = env_config['save_path']

    now = datetime.now()
    datestr = now.strftime('%m|%d-%H:%M:%S')
    datestr = datestr          

    paths = [
        f'./pretrained/{dir_path}/best_{datestr}.pt',
        f'./results/{dir_path}/{datestr}.csv',
    ]

    for path in paths:
        dirname = os.path.dirname(path)
        Path(dirname).mkdir(parents=True, exist_ok=True)

    return paths

def get_score(test_result, val_result):

    feature_num = len(test_result[0][0])
    np_test_result = np.array(test_result)
    np_val_result = np.array(val_result)

    test_labels = np_test_result[2, :, 0].tolist()

    test_scores, normal_scores = get_full_err_scores(test_result, val_result)

    top1_best_info = get_best_performance_data(test_scores, test_labels, topk=1) 
    top1_val_info = get_val_performance_data(test_scores, normal_scores, test_labels, topk=1)


    print('=========================** Result **============================\n')

    info = None
    if env_config['report'] == 'best':
        info = top1_best_info
    elif env_config['report'] == 'val':
        info = top1_val_info

    print(f'F1 score: {info[0]}')
    print(f'precision: {info[1]}')
    print(f'recall: {info[2]}\n')

In [21]:
if len(env_config['load_model_path']) > 0:
    model_save_path = env_config['load_model_path']
else:
    model_save_path = get_save_path()[0]

    train_log = train_model(model, model_save_path, 
        config = train_config,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader, 
        feature_map=feature_map,
        test_dataloader=test_dataloader,
        test_dataset=test_dataset,
        train_dataset=train_dataset,
        dataset_name=env_config['dataset']
    )

# test            
model.load_state_dict(torch.load(model_save_path))
best_model = model.to(device)

_, test_result = test_model(best_model, test_dataloader)
_, val_result = test_model(best_model, val_dataloader)

get_score(test_result, val_result)

epoch (0 / 50) (Loss:0.00650947, ACU_loss:1.73802973)
epoch (1 / 50) (Loss:0.00619083, ACU_loss:1.65295036)
epoch (2 / 50) (Loss:0.00653698, ACU_loss:1.74537246)
epoch (3 / 50) (Loss:0.00612448, ACU_loss:1.63523712)
epoch (4 / 50) (Loss:0.00590560, ACU_loss:1.57679584)
epoch (5 / 50) (Loss:0.00584417, ACU_loss:1.56039281)
epoch (6 / 50) (Loss:0.00658256, ACU_loss:1.75754244)
epoch (7 / 50) (Loss:0.00669261, ACU_loss:1.78692775)
epoch (8 / 50) (Loss:0.00667318, ACU_loss:1.78174033)
epoch (9 / 50) (Loss:0.00629843, ACU_loss:1.68168049)
epoch (10 / 50) (Loss:0.00741544, ACU_loss:1.97992261)
epoch (11 / 50) (Loss:0.00795506, ACU_loss:2.12400040)
epoch (12 / 50) (Loss:0.00739637, ACU_loss:1.97483079)
epoch (13 / 50) (Loss:0.00696771, ACU_loss:1.86037868)


/tmp/ipykernel_2946356/4178983472.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path))


=========================** Result **============================

F1 score: 0.7807017543859649
precision: 0.8492734764693125
recall: 0.7225092250922509

